In [178]:
import os
import json
from langchain_groq import ChatGroq
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

from langchain_chroma import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.tools import DuckDuckGoSearchRun 

from dotenv import load_dotenv
load_dotenv()

from pathlib import Path

In [179]:
STYLED_RAW_DIR = Path('../data/styled/styled_raw.json')
STYLED_SECTION_DIR = Path('../data/styled/styled_sections.json')

In [180]:
# setup the groq llm
llm = ChatGroq(
    model='llama-3.3-70b-versatile',
    temperature=0.3,
)

In [181]:
# load the writing style
with open(STYLED_RAW_DIR, 'r', encoding='utf-8') as f: 
    styled_raw = json.load(f) 
    
with open(STYLED_SECTION_DIR, 'r', encoding='utf-8') as f: 
    styled_section = json.load(f)

In [182]:
styled_raw

{'122516956_CaseStudy_EEI5270.pdf': "Here's a rewritten version of the provided text in a more readable and grammatically correct tone:\n\n**Case Study: EEI Information Security - LBD Mawijesundara Spa**\n\n**Table of Contents**\n\n1. **Background and Threat Assessment**\n2. **Quantum Computing and Cryptography**\n\t* **Quantum Computing**: Understanding the basics of quantum computing and its potential impact on cryptography.\n\t* **Quantum Cryptography**: Exploring the relevance of quantum cryptography for developing nations.\n3. **Security Analysis of Thaproban**\n\t* **Key Vulnerabilities**: Identifying vulnerabilities in Thaproban's current systems.\n\t* **Systematic Issues**: Analyzing systematic issues that amplify the risk of a quantum attack.\n\t* **Consequences of a Quantum Attack**: Understanding the potential consequences of a quantum attack on Thaproban.\n4. **PQ Landscape**\n\t* **Recent Advances in PQC**: Discussing recent advances in post-quantum cryptography (PQC).\n\t

In [183]:
styled_section 

{'122516956_CaseStudy_EEI5270.pdf': {'unknown': '<think>\nOkay, the user wants me to rewrite the given text in their own grammar and tone while keeping the meaning the same. Let me look at the original text first. It\'s a list of sections for a case study on information security, specifically about quantum computing and cryptography. The original is a bit jumbled, with technical terms and section titles all in one line.\n\nFirst, I need to parse the original content into individual sections. The user probably wants a structured table of contents but in a more readable format. The original has phrases like "background and threat assessment", "quantum computing and cryptography", etc. My task is to rephrase these into clear, concise headings without losing the original intent.\n\nI should check if there are any repeated terms or sections that can be grouped. For example, "quantum computing" and "quantum cryptography" are separate but related. The user might want them as distinct sections

In [184]:
# Build a global style summary from styled_raw
if styled_raw:
    style_examples = "\n\n".join(list(styled_raw.values())[:10])  # few samples
else:
    style_examples = "Write in a clear academic style with structured arguments."

In [185]:
# load the vectorstore
embedding_fn = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

vectorstore = Chroma(
    persist_directory='../vectorstore', 
    embedding_function=embedding_fn
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 5}) 

In [186]:
# # tools
# # 1 - styled_raw function
# def query_styled_raw(query: str) -> str:
#     '''Search across whole styled documents'''
#     results = []
#     for fname, text in styled_raw.items():
#         if query.lower() in text.lower():
#             results.append(f'[{fname}] {text[:400]}...')
#     return '\n'.join(results) if results else 'No relevant styled raw document found.'

# # 2 - styled section functoin
# def query_styled_sections(query: str) -> str: 
#     '''Search across styled sections'''
#     results = []
#     for fname, sections in styled_section.items():
#         for section, text in section.items(): 
#             if query.lower() in text.lower(): 
#                 results.append(f'[{fname}::{section}] {text[:400]}...')
#     return '\n'.join(results) if results else 'No relevant styled section found'

# # web search
# web_search = DuckDuckGoSearchRun() 

# tools = [
#     Tool(
#         name='Vector Rag',
#         func=lambda q: retriever.get_relevant_documents(q),
#         description='retrieve academic context from vector databases'
#     ),
#     Tool(
#         name='Styled Raw Lookup',
#         func=query_styled_raw,
#         description='retrieved styled academic writing from full raw documents' 
#     ), 
#     Tool(
#         name='Styled Section Lookup',
#         func=query_styled_sections,
#         description='Retrieve styled academic writing from section level documents'
#     ),
#     Tool(
#         name='Web Search',
#         func=web_search.run,
#         description='Search the web for latest information'
#     )
# ]


In [187]:
# RAG + styled context
def rag_with_style(question: str): 
    '''Retrieve from vector DB + styled raw examples'''
    docs = retriever.get_relevant_documents(question)
    context = '\n\n'.join([doc.page_content for doc in docs])
    
    # Use fallback text if empty
    context_text = context if context else "No additional context available."
    style_examples_text = style_examples if style_examples else "No student style examples provided."
    
    prompt_template = '''
    You are an academic assistant.
    ALWAYS USE this Tool eventhough you can answer without them:
    1. Studen't styled examples
    2. Retrieved academic context
    to answer the question. 
    
    Styled Examples:
    {styled_examples}
    
    Question:
    {question}
    
    Retrieved Context:
    {context}
    
    Write a final academic answer that matches the student's style.
    '''
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=['question', 'context', 'styled_examples']
    )
    
    formatted_prompt = prompt.format(
        question=question,
        context=context_text,
        styled_examples=style_examples_text
    )
    
    response = llm.invoke(formatted_prompt)
    return response.content

In [188]:
# web search tool
web_search = DuckDuckGoSearchRun()
def web_search_tool(query: str): 
   return web_search.run(query) 

# # Grammar checker tool
# def grammar_check(text: str):
#     prompt = f'''
#     You are an academic editor.
#     improve grammar, clarity and flow of the follwing text,
#     while preserving the  student's academic style.
    
#     Styled Examples:
#     {style}
    
#     Text:
#     {text}
#     '''
    
#     response = llm.invoke(prompt)
#     return response.content


In [189]:
# tools
tools = [
    Tool(
        name="AcademicRAG",
        func=rag_with_style,
        description="Always use this to answer knowledge queries using the RAG method."
    ),
    Tool(
        name="ResearchTool",
        func=web_search_tool,
        description="Search the web for academic information"
    ),
    # Tool(
    #     name="GrammarChecker",
    #     func=grammar_check,
    #     description="Improve grammar and clarity while preserving style"
    # ),
]

In [190]:
# memeory
memory = ConversationBufferMemory(
    memory_key='chat_history', 
    return_messages=True
)

# Agent
team_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
)

In [192]:
query = "what is the role of engineers in the current world?"
response = team_agent.run(query)
print("Team Agent Response:\n", response)



> Entering new AgentExecutor chain...
To answer this question, I need to consider the various fields and industries where engineers play a crucial role. Engineers are involved in designing, developing, and implementing solutions to real-world problems, and their work has a significant impact on society.

Action: AcademicRAG
Action Input: What are the roles and responsibilities of engineers in modern society?
Observation: **The Roles and Responsibilities of Engineers in Modern Society**

Engineers play a vital role in modern society, and their responsibilities are multifaceted. As problem-solvers, engineers are tasked with designing, developing, and implementing innovative solutions to real-world problems. Their work has a significant impact on various aspects of our lives, from the infrastructure we use to the technologies that drive our economy.

**Background and Context**

In today's complex and interconnected world, engineers are faced with numerous challenges that require creativ

c:\Users\madhu\anaconda3\envs\tf\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: In order to qualify and enroll in the Fee-Free Overdraft feature, you must receive a minimum of $200 or more in Eligible … In order to qualify and enroll in the Fee-Free Overdraft feature, you must receive a minimum of $200 or more in Eligible … With a Current account, you can get a debit card or the Build Card to safely build credit, all while having access to features like … Apr 29, 2025 · You have just one balance to view and manage on Current’s platform (no moving money between spending and … In order to qualify and enroll in the Fee-Free Overdraft feature, you must receive a minimum of $200 or more in Eligible …
Thought:It seems like the ResearchTool output is not relevant to the question about the current trends and challenges in engineering. The output appears to be related to a banking or financial service.

Action: ResearchTool
Action Input: engineering trends and challenges 2024

c:\Users\madhu\anaconda3\envs\tf\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: No good DuckDuckGo Search Result was found
Thought:It seems like the ResearchTool output is still not relevant to the question about the current trends and challenges in engineering. The output appears to be an error message or an empty result.

Action: AcademicRAG
Action Input: What are the current trends and challenges in engineering?
Observation: **Current Trends and Challenges in Engineering: An Overview**

The field of engineering is constantly evolving, with new trends and challenges emerging every year. As technology advances and societal needs change, engineers must adapt and innovate to stay ahead of the curve. In this answer, we will explore some of the current trends and challenges in engineering, highlighting key areas of focus and potential solutions.

**Trends in Engineering**

1. **Sustainability and Environmental Engineering**: With growing concerns about climate change and environmental degradation, sustainable engineering practices are becoming increasin